<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2025notebooks/2025_0331ca_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 藤本先生からいただいた対応分析を Python で再実装する試み

* Date: 2025_0331
* Author: 浅川伸一

## 0.1 下準備 データファイルの読み込み

In [ ]:
excel_fname = "失語6例_TLPA呼称データ_241212_藤本先生コラボ用_送信版.xlsx"

# Google Colab で実行する場合，データのエクセルファイルをアップロードする必要がある。
import IPython
isColab = 'google.colab' in str(IPython.get_ipython())
if isColab:
    from google.colab import files
    uploaded = files.upload()

In [ ]:
# 必要となるライブラリのインストール
try:
    import mca
except ImportError:
    !pip install mca
    import mca

import matplotlib.pyplot as plt
try:
    import japanize_matplotlib
except:
    !pip install japanize_matplotlib
    import japanize_matplotlib

# 1. データファイルの読み込み

In [ ]:
import numpy as np
import pandas as pd

excel_fname = "失語6例_TLPA呼称データ_241212_藤本先生コラボ用_送信版.xlsx"

# 藤本先生の R コードでは d0 という名前でデータを扱っていた。そのためここでも同じ変数名 d0 を用いる。
d0 = pd.read_excel(excel_fname)[['カテゴリー','症例A', '症例B', '症例C', '症例D', '症例E', '症例F']]

# 反応の型を定義している部分を抜き出す
resp_types = pd.read_excel(excel_fname)[['Unnamed: 15', 'Unnamed: 16']].to_numpy()[1:9]
resps = {i[0]:i[1] for i in resp_types}

print(f'反応の種類:{resps}')
print(f'刺激図版の種類:{d0.カテゴリー.unique()}')

stim_cats = {c:i+1 for i, c in enumerate(d0.カテゴリー.unique())}
print(f'刺激図版のカテゴリー番号:{stim_cats}')

# 刺激図版のカテゴリを数値に変換して格納
stim_cat_nums = [stim_cats[x] for x in d0.カテゴリー.to_numpy()]
pd.options.mode.copy_on_write = True
d0['stim_cat_nums'] = stim_cat_nums

# 読み込んだデータ, pandas のデータフレームに格納してある。そのデータを表示。
d0

In [ ]:
# 各症例ごとに反応を集計
for case in ['症例A', '症例B', '症例C', '症例D', '症例E', '症例F']:
    print(pd.crosstab(d0.stim_cat_nums, d0.eval(case)), end="\n---\n")
    break

# 2. 対応分析の実施

In [ ]:
# 症例 A を取り出して対応分析実施

d1 = pd.crosstab(d0.stim_cat_nums, d0.症例A)
mca_counts = mca.MCA(d1, benzecri=False)

print(f'行に関する因子得点\n{mca_counts.fs_r(N=2)}')
print(f'列に関する因子得点\n{mca_counts.fs_c(N=2)}')
d1

In [ ]:
rows = mca_counts.fs_r(N=2)
cols = mca_counts.fs_c(N=2)

plt.scatter( rows[:,0], rows[:,1], marker="None")
labels = d0.カテゴリー
labels = [i for i in stim_cats]
for label,x,y in zip(labels,rows[:,0],rows[:,1]):
    plt.annotate(label,xy = (x, y), c="b")

plt.scatter(cols[:, 0], cols[:, 1], marker="None")
#labels = df.columns
labels = [resps[c] for c in pd.crosstab(d0.stim_cat_nums, d0.症例A).columns]
for label, x, y in zip(labels, cols[:, 0], cols[:, 1]):
    plt.annotate(label, xy=(x, y), c="r")

# 3. 同じデータに対して PCA

In [8]:
# 下準備
try:
    import seaborn as sns
except ImportError:
    !pip install --upgrade seaborn
    import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
X = StandardScaler().fit_transform(d1.to_numpy())
pca = PCA(n_components=2).fit(X)
X_reduced = pca.transform(X)

loadings = pca.components_[:2].T
pvars = pca.explained_variance_ratio_[:2] * 100
arrows = loadings * np.ptp(X_reduced, axis=0)
width = -0.0075 * np.min([np.subtract(*plt.xlim()), np.subtract(*plt.ylim())])

plt.scatter(X_reduced[:,0], X_reduced[:,1])
for label,x,y in zip(stim_cats, X_reduced[:,0],X_reduced[:,1]):
    plt.annotate(label, xy = (x, y), c="b")
# Plot arrows.
horizontal_alignment = ['right', 'left', 'right', 'right']
vertical_alignment = ['bottom', 'top', 'top', 'bottom']
for (i, arrow), ha, va in zip(enumerate(arrows), horizontal_alignment, vertical_alignment):
    plt.arrow(0, 0, *arrow, color='k', alpha=0.5, width=width, ec='none',
              length_includes_head=True)
    plt.text(*(arrow * 1.05), [x[1] for x in resp_types][i], ha=ha, va=va,
    #plt.text(*(arrow * 1.05), list(stim_cats.keys())[i], ha=ha, va=va,
             fontsize='large', color='green')
